First let's import important libraries and packages

In [6]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from PIL import Image

Implimentation of the Unet Model

In [3]:
def unet_model(input_shape=(256, 256, 3)):
    inputs = keras.Input(shape=input_shape)
    
    # Contracting path
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = keras.layers.Dropout(0.5)(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(drop4)
    
    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = keras.layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = keras.layers.Dropout(0.5)(conv5)
    
   # Expanding path
    up6 = keras.layers.Conv2DTranspose(512, 2, strides=(2, 2), padding='same')(drop5)
    merge6 = keras.layers.concatenate([drop4, up6], axis=3)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = keras.layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = keras.layers.Conv2DTranspose(256, 2, strides=(2, 2), padding='same')(conv6)
    merge7 = keras.layers.concatenate([conv3, up7], axis=3)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = keras.layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = keras.layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv7)
    merge8 = keras.layers.concatenate([conv2, up8], axis=3)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = keras.layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = keras.layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv8)
    merge9 = keras.layers.concatenate([conv1, up9], axis=3)
    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = keras.layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = keras.layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model

Training the Model

In [14]:
def train_model():
    #train the model
    images_path = "/content/drive/MyDrive/dataset/images/train"
    mask_path = "/content/drive/MyDrive/dataset/annotations/train"

    image_files = [f for f in os.listdir(images_path)]
    

    X_train, Y_train = [], []

    for image_file in image_files:

      img = cv2.imread(os.path.join(images_path, image_file))
      if img is not None:
        img = cv2.resize(img, (256,256))
        mask = cv2.imread(os.path.join(mask_path, image_file.split(".")[0]+"_mask.png"))
        mask = cv2.resize(mask, (256, 256))
        X_train.append(img)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)[1]
        mask=np.expand_dims(mask,axis=-1)
        Y_train.append(mask)

    X_train = np.array(X_train)
    Y_train = np.array(Y_train)
    
    model = unet_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics= ["accuracy"])
    model.fit(X_train, Y_train, batch_size=16, epochs=25, validation_split=0.2)


In [15]:
train_model()

Epoch 1/25
20/20 [==============================] - 54s 2s/step - loss: 5.4264 - accuracy: 0.6411 - val_loss: 0.0826 - val_accuracy: 0.9711
Epoch 2/25
20/20 [==============================] - 23s 1s/step - loss: 0.0686 - accuracy: 0.9803 - val_loss: 0.0340 - val_accuracy: 0.9894
Epoch 3/25
20/20 [==============================] - 20s 1s/step - loss: 0.0282 - accuracy: 0.9903 - val_loss: 0.0218 - val_accuracy: 0.9924
Epoch 4/25
20/20 [==============================] - 20s 1s/step - loss: 0.0223 - accuracy: 0.9921 - val_loss: 0.0180 - val_accuracy: 0.9935
Epoch 5/25
20/20 [==============================] - 20s 1s/step - loss: 0.0196 - accuracy: 0.9931 - val_loss: 0.0193 - val_accuracy: 0.9932
Epoch 6/25
20/20 [==============================] - 21s 1s/step - loss: 0.0179 - accuracy: 0.9940 - val_loss: 0.0143 - val_accuracy: 0.9949
Epoch 7/25
20/20 [==============================] - 21s 1s/step - loss: 0.0154 - accuracy: 0.9950 - val_loss: 0.0132 - val_accuracy: 0.9953
Epoch 8/25
20/20 [==

Testing the Model

In [10]:
images_path = "/content/drive/MyDrive/dataset/images/test"
mask_path = "/content/drive/MyDrive/dataset/annotations/test"

# Get a list of all the image files in the test images directory
image_files = os.listdir(images_path)

# Create lists to store the test images and masks
X_test = []
Y_test = []

# Loop through the test images and masks and preprocess them
for image_file in image_files:
    # Load the image
    img = cv2.imread(os.path.join(images_path, image_file))
    if img is not None:
        img = cv2.resize(img, (256,256))
        
        # Load and preprocess the mask
        mask = cv2.imread(os.path.join(mask_path, image_file.split(".")[0]+"_mask.png"))
        mask = cv2.resize(mask, (256, 256))
        
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)[1]
        mask = np.expand_dims(mask, axis=-1)
        
        # Add the image and mask to the test lists
        X_test.append(img)
        Y_test.append(mask)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

model = unet_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics= ["accuracy"])

Y_pred = model.predict(X_test)

test_loss, test_accuracy = model.evaluate(X_test, Y_test)

print(test_loss)
print(test_accuracy)




# # Convert the predicted masks to the same data type as the Y_test masks
# Y_pred = np.round(Y_pred).astype(np.uint8)


# # Calculate the dice coefficient for each image in the test set
# dice_scores = []
# for i in range(len(Y_test)):
#     dice = dice_coef(Y_test[i], Y_pred[i])
#     dice_scores.append(dice)

# # Calculate the average dice coefficient for the test set
# average_dice = np.mean(dice_scores)

# print("Average dice coefficient:", average_dice)





2/2 [==============================] - 1s 308ms/step - loss: 1.5186 - accuracy: 0.6515
1.5185644626617432
0.6514761447906494
